In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm

sns.set(font=fm.FontProperties(fname='/System/Library/Fonts/PingFang.ttc').get_name())

%matplotlib inline

### 讀入資料集

In [43]:

train_path = './30_Training Dataset_V2/training_data.csv'
train = pd.read_csv(train_path)
print("Full train dataset shape is {}".format(train.shape))

public_test_path = './30_Public Dataset_Public Sumission Template_v2/public_dataset.csv'
public_test = pd.read_csv(public_test_path)
print("Full test dataset shape is {}".format(public_test.shape))

private_test_path = './30_Private Dataset _Private and Publict Submission Template_v2/private_dataset.csv'
private_test = pd.read_csv(private_test_path)
print("Full test dataset shape is {}".format(private_test.shape))

public_test_ID = public_test['ID']
private_test_ID = private_test['ID']

Full train dataset shape is (11751, 22)
Full test dataset shape is (5876, 21)
Full test dataset shape is (5875, 21)


檢查資料型別

In [44]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11751 entries, 0 to 11750
Data columns (total 22 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      11751 non-null  object 
 1   縣市      11751 non-null  object 
 2   鄉鎮市區    11751 non-null  object 
 3   路名      11751 non-null  object 
 4   土地面積    11751 non-null  float64
 5   使用分區    11751 non-null  object 
 6   移轉層次    11751 non-null  int64  
 7   總樓層數    11751 non-null  int64  
 8   主要用途    11751 non-null  object 
 9   主要建材    11751 non-null  object 
 10  建物型態    11751 non-null  object 
 11  屋齡      11751 non-null  float64
 12  建物面積    11751 non-null  float64
 13  車位面積    11751 non-null  float64
 14  車位個數    11751 non-null  float64
 15  橫坐標     11751 non-null  int64  
 16  縱坐標     11751 non-null  int64  
 17  備註      92 non-null     object 
 18  主建物面積   11751 non-null  float64
 19  陽台面積    11751 non-null  float64
 20  附屬建物面積  11751 non-null  float64
 21  單價      11751 non-null  float64
dty

資料說明中已去識別化的欄位
* 土地面積
* 建物面積
* 車位面積
* 橫坐標
* 縱坐標
* 主建物面積
* 陽台面積
* 附屬建物面積
* 單價

In [45]:
train['車位個數'] = train['車位個數'].astype('int64')

In [46]:
""" numeric_features = train.select_dtypes(include=['int64', 'float64']).columns.drop(['橫坐標', '縱坐標'])

fig, axs = plt.subplots(4, 3, figsize=(15, 15))
for i, feature in enumerate(numeric_features):
    row = i // 3
    col = i % 3
    sns.kdeplot(train[feature], fill=True, ax=axs[row][col])
plt.show() """


" numeric_features = train.select_dtypes(include=['int64', 'float64']).columns.drop(['橫坐標', '縱坐標'])\n\nfig, axs = plt.subplots(4, 3, figsize=(15, 15))\nfor i, feature in enumerate(numeric_features):\n    row = i // 3\n    col = i % 3\n    sns.kdeplot(train[feature], fill=True, ax=axs[row][col])\nplt.show() "

In [47]:
""" from matplotlib.cm import coolwarm
from matplotlib.colors import Normalize

fig, ax = plt.subplots()
scatter = ax.scatter(train['橫坐標'], train['縱坐標'], c=train['單價'], cmap=coolwarm, norm=Normalize())
plt.colorbar(scatter)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_title('Scatter plot of coordinates colored by target variable')
ax.set_aspect('equal', adjustable='box')
plt.show() """


" from matplotlib.cm import coolwarm\nfrom matplotlib.colors import Normalize\n\nfig, ax = plt.subplots()\nscatter = ax.scatter(train['橫坐標'], train['縱坐標'], c=train['單價'], cmap=coolwarm, norm=Normalize())\nplt.colorbar(scatter)\nax.set_xlabel('X')\nax.set_ylabel('Y')\nax.set_title('Scatter plot of coordinates colored by target variable')\nax.set_aspect('equal', adjustable='box')\nplt.show() "

In [48]:
""" lower_limit, upper_limit = np.percentile(a=train['單價'],q=[1,99])
print(upper_limit)
print(lower_limit)
price_filtered = train['單價'][(train['單價']>lower_limit) & (train['單價']<upper_limit)].index

fig, ax = plt.subplots()
scatter = ax.scatter(train['橫坐標'][price_filtered], train['縱坐標'][price_filtered], c=train['單價'][price_filtered], cmap=coolwarm, norm=Normalize())
plt.colorbar(scatter)
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_title('Scatter plot of coordinates colored by target variable')
ax.set_aspect('equal', adjustable='box')
plt.show() """


" lower_limit, upper_limit = np.percentile(a=train['單價'],q=[1,99])\nprint(upper_limit)\nprint(lower_limit)\nprice_filtered = train['單價'][(train['單價']>lower_limit) & (train['單價']<upper_limit)].index\n\nfig, ax = plt.subplots()\nscatter = ax.scatter(train['橫坐標'][price_filtered], train['縱坐標'][price_filtered], c=train['單價'][price_filtered], cmap=coolwarm, norm=Normalize())\nplt.colorbar(scatter)\nax.set_xlabel('X')\nax.set_ylabel('Y')\nax.set_title('Scatter plot of coordinates colored by target variable')\nax.set_aspect('equal', adjustable='box')\nplt.show() "

### 特徵編碼、缺失值處理

同時處理 `train` & `test` 資料

In [49]:
all_data = pd.concat([train.drop(['ID', '單價'], axis=1 ), public_test.drop(['ID'],axis=1), private_test.drop(['ID'],axis=1)])
all_data.reset_index(drop=True, inplace=True)
all_data.shape

(23502, 20)

In [50]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23502 entries, 0 to 23501
Data columns (total 20 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   縣市      23502 non-null  object 
 1   鄉鎮市區    23502 non-null  object 
 2   路名      23502 non-null  object 
 3   土地面積    23502 non-null  float64
 4   使用分區    23502 non-null  object 
 5   移轉層次    23502 non-null  int64  
 6   總樓層數    23502 non-null  int64  
 7   主要用途    23502 non-null  object 
 8   主要建材    23502 non-null  object 
 9   建物型態    23502 non-null  object 
 10  屋齡      23502 non-null  float64
 11  建物面積    23502 non-null  float64
 12  車位面積    23502 non-null  float64
 13  車位個數    23502 non-null  float64
 14  橫坐標     23502 non-null  int64  
 15  縱坐標     23502 non-null  int64  
 16  備註      182 non-null    object 
 17  主建物面積   23502 non-null  float64
 18  陽台面積    23502 non-null  float64
 19  附屬建物面積  23502 non-null  float64
dtypes: float64(8), int64(4), object(8)
memory usage: 3.6+ MB


### `baseline model` : Only One-hot encoding
#### `MAPE`: 10.5002%

處理缺失值: `備註`

In [51]:
all_data['備註'] = all_data['備註'].apply(lambda x: False if pd.isnull(x) or x == '' else True)

One-hot Encoding

In [52]:
all_data_OH = pd.get_dummies(all_data)
all_data_OH.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23502 entries, 0 to 23501
Columns: 4071 entries, 土地面積 to 建物型態_透天厝
dtypes: bool(1), float64(8), int64(4), uint8(4058)
memory usage: 93.1 MB


In [53]:
X_OH = all_data_OH[:train.shape[0]]
y_OH = train['單價']
print("X shape is {}".format(X_OH.shape))


X shape is (11751, 4071)


In [54]:
#%%time

from sklearn.model_selection import cross_val_score
import xgboost as xgb

""" baseline = xgb.XGBRegressor(random_state=42)

baseline_score = cross_val_score(baseline, X_OH, y_OH, scoring='neg_mean_absolute_percentage_error', cv=5)

print(baseline_score)

baseline_score = -1 * baseline_score.mean()

print("MAPE: %.4f%%" % (baseline_score * 100.0)) """


' baseline = xgb.XGBRegressor(random_state=42)\n\nbaseline_score = cross_val_score(baseline, X_OH, y_OH, scoring=\'neg_mean_absolute_percentage_error\', cv=5)\n\nprint(baseline_score)\n\nbaseline_score = -1 * baseline_score.mean()\n\nprint("MAPE: %.4f%%" % (baseline_score * 100.0)) '

### 特徵編碼

In [55]:
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23502 entries, 0 to 23501
Data columns (total 20 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   縣市      23502 non-null  object 
 1   鄉鎮市區    23502 non-null  object 
 2   路名      23502 non-null  object 
 3   土地面積    23502 non-null  float64
 4   使用分區    23502 non-null  object 
 5   移轉層次    23502 non-null  int64  
 6   總樓層數    23502 non-null  int64  
 7   主要用途    23502 non-null  object 
 8   主要建材    23502 non-null  object 
 9   建物型態    23502 non-null  object 
 10  屋齡      23502 non-null  float64
 11  建物面積    23502 non-null  float64
 12  車位面積    23502 non-null  float64
 13  車位個數    23502 non-null  float64
 14  橫坐標     23502 non-null  int64  
 15  縱坐標     23502 non-null  int64  
 16  備註      23502 non-null  bool   
 17  主建物面積   23502 non-null  float64
 18  陽台面積    23502 non-null  float64
 19  附屬建物面積  23502 non-null  float64
dtypes: bool(1), float64(8), int64(4), object(7)
memory usage: 3.4+ MB


In [56]:
district_names = all_data[['縣市', '鄉鎮市區']].apply(lambda x: ''.join(x), axis=1)
district_names.unique().shape

(137,)

#### Label Encoding: `縣市`

In [57]:
all_data['縣市'].value_counts()

新北市    8621
台北市    4828
高雄市    2643
桃園市    2302
台中市    2042
台南市     903
新竹縣     694
新竹市     517
基隆市     440
宜蘭縣     138
苗栗縣      95
屏東縣      82
嘉義市      64
金門縣      63
彰化縣      40
花蓮縣      23
雲林縣       5
嘉義縣       2
Name: 縣市, dtype: int64

In [58]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
all_data['縣市'] = le.fit_transform(all_data['縣市'])

all_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23502 entries, 0 to 23501
Data columns (total 20 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   縣市      23502 non-null  int64  
 1   鄉鎮市區    23502 non-null  object 
 2   路名      23502 non-null  object 
 3   土地面積    23502 non-null  float64
 4   使用分區    23502 non-null  object 
 5   移轉層次    23502 non-null  int64  
 6   總樓層數    23502 non-null  int64  
 7   主要用途    23502 non-null  object 
 8   主要建材    23502 non-null  object 
 9   建物型態    23502 non-null  object 
 10  屋齡      23502 non-null  float64
 11  建物面積    23502 non-null  float64
 12  車位面積    23502 non-null  float64
 13  車位個數    23502 non-null  float64
 14  橫坐標     23502 non-null  int64  
 15  縱坐標     23502 non-null  int64  
 16  備註      23502 non-null  bool   
 17  主建物面積   23502 non-null  float64
 18  陽台面積    23502 non-null  float64
 19  附屬建物面積  23502 non-null  float64
dtypes: bool(1), float64(8), int64(5), object(6)
memory usage: 3.4+ MB


#### Hash Encoding: `鄉鎮市區`, n_features=128

In [59]:
from sklearn.feature_extraction import FeatureHasher

hasher = FeatureHasher(n_features=128, input_type='string')

hashed_features = hasher.transform(district_names.values.reshape(-1, 1))

hashed_features_df = pd.DataFrame(hashed_features.toarray()).add_prefix('鄉鎮市區_')

# 將哈希編碼的特徵加到原始DataFrame中
all_data = pd.concat([all_data, hashed_features_df], axis=1)
all_data.drop(['鄉鎮市區'], axis=1, inplace=True)

all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23502 entries, 0 to 23501
Columns: 147 entries, 縣市 to 鄉鎮市區_127
dtypes: bool(1), float64(136), int64(5), object(5)
memory usage: 26.2+ MB


#### Hash Encoding: `路名`

In [60]:
all_data['路名'].unique().shape

(3881,)

In [61]:
road_names = all_data[['路名']]

hasher = FeatureHasher(n_features=256, input_type='string')

hashed_features = hasher.transform(road_names.values)

hashed_features_df = pd.DataFrame(hashed_features.toarray()).add_prefix('路名_')

# 將哈希編碼的特徵加到原始DataFrame中
all_data = pd.concat([all_data, hashed_features_df], axis=1)
all_data.drop(['路名'], axis=1, inplace=True)

all_data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23502 entries, 0 to 23501
Columns: 402 entries, 縣市 to 路名_255
dtypes: bool(1), float64(392), int64(5), object(4)
memory usage: 71.9+ MB


#### One-hot Encoding : `使用分區`

In [62]:
all_data['使用分區'].value_counts()

None    22318
住         652
其他        260
商         163
農          64
工          45
Name: 使用分區, dtype: int64

In [63]:
#all_data['使用分區'] = all_data.groupby('使用分區')['使用分區'].transform('count')

### 測試其他編碼方法不會降低MAPE

all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23502 entries, 0 to 23501
Columns: 402 entries, 縣市 to 路名_255
dtypes: bool(1), float64(392), int64(5), object(4)
memory usage: 71.9+ MB


#### One-hot Encoding : `主要用途`

In [64]:
all_data['主要用途'].value_counts()

住家用      16400
集合住宅      5343
其他         978
商業用        490
一般事務所      154
國民住宅        57
住商用         24
辦公室         21
工業用         19
住工用          9
店鋪           6
廠房           1
Name: 主要用途, dtype: int64

In [65]:
#le = LabelEncoder()
#all_data['主要用途'] = le.fit_transform(all_data['主要用途'])
### 測試其他編碼方法不會降低MAPE

all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23502 entries, 0 to 23501
Columns: 402 entries, 縣市 to 路名_255
dtypes: bool(1), float64(392), int64(5), object(4)
memory usage: 71.9+ MB


#### One-hot Encoding : `主要建材`

In [66]:
all_data['主要用途'].value_counts()

住家用      16400
集合住宅      5343
其他         978
商業用        490
一般事務所      154
國民住宅        57
住商用         24
辦公室         21
工業用         19
住工用          9
店鋪           6
廠房           1
Name: 主要用途, dtype: int64

In [67]:
### 測試其他編碼方法不會降低MAPE
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23502 entries, 0 to 23501
Columns: 402 entries, 縣市 to 路名_255
dtypes: bool(1), float64(392), int64(5), object(4)
memory usage: 71.9+ MB


#### One-hot Encoding : `建物型態`

In [68]:
all_data['建物型態'].value_counts()

住宅大樓(11層含以上有電梯)    14271
公寓(5樓含以下無電梯)        4911
華廈(10層含以下有電梯)       4308
透天厝                   12
Name: 建物型態, dtype: int64

In [69]:
### 測試其他編碼方法不會降低MAPE
all_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23502 entries, 0 to 23501
Columns: 402 entries, 縣市 to 路名_255
dtypes: bool(1), float64(392), int64(5), object(4)
memory usage: 71.9+ MB


In [70]:
all_data = pd.get_dummies(all_data)

### `model 2` : Label + hash + One-hot encoding

#### `MAPE`: 10.3498%

In [71]:
X = all_data[:train.shape[0]]
X_test = all_data[train.shape[0]:]
y = train["單價"]
print("X shape is {}".format(X.shape))
print("X_test shape is {}".format(X_test.shape))

X shape is (11751, 426)
X_test shape is (11751, 426)


In [72]:
""" %%time

model2 = xgb.XGBRegressor(random_state=42)
model2_score = cross_val_score(model2, X, y, scoring='neg_mean_absolute_percentage_error', cv=5)

print(model2_score)

model2_score = -1 * model2_score.mean()

print("MAPE: %.4f%%" % (model2_score * 100.0)) """

' %%time\n\nmodel2 = xgb.XGBRegressor(random_state=42)\nmodel2_score = cross_val_score(model2, X, y, scoring=\'neg_mean_absolute_percentage_error\', cv=5)\n\nprint(model2_score)\n\nmodel2_score = -1 * model2_score.mean()\n\nprint("MAPE: %.4f%%" % (model2_score * 100.0)) '

### 新增特徵

#### 座標轉換

In [73]:
#### 座標轉換
from pyproj import Transformer

# Create a transformer object for TWD97 to WGS84 conversion
transformer = Transformer.from_crs("epsg:3826", "epsg:4326", always_xy=True)

coordinates = pd.DataFrame()

# Convert TWD97 coordinates to WGS84
coordinates['lng'], coordinates['lat'] = transformer.transform(
    all_data['橫坐標'].values, all_data['縱坐標'].values
)

# Now the all_data should have two new columns with the converted WGS84 latitude and longitude
print("all_data shape :", all_data.shape)
print("coordinates shape :",coordinates.shape)

coordinates.head()

all_data shape : (23502, 426)
coordinates shape : (23502, 2)


,lng,lat
0,121.547608,25.022469
1,121.502124,25.019127
2,120.365799,22.640966
3,121.462402,25.058663
4,121.469444,25.023585


In [74]:
from sklearn.neighbors import BallTree

def count_facilities(tree, lat, lng, km):
    radius = km * 1 / 6371  # 1 km in radians
    indices = tree.query_radius(np.deg2rad([[lat, lng]]), r=radius)
    return len(indices[0])

def distance_to_nearest_facilities(tree, lat, lng, k = 1):
    # 查詢最近點
    dist, _ = tree.query(np.deg2rad([[lat, lng]]), k)
    # 將弧度轉換為公里
    return dist[0][0] * 6371

def average_distance_to_nearest_facilities(tree, lat, lng, k=5):
    # 查詢最近的 k 個鄰居，這裡 k=5
    dist, _ = tree.query(np.deg2rad([[lat, lng]]), k=k)
    # 將弧度轉換為公里並計算平均距離
    return np.mean(dist[0]) * 6371

#### `大學` : 5, 10 KM 內數量, 最近距離, 最近5平均距離

In [75]:
# 載入大學基本資料
university_data = pd.read_csv('30_Training Dataset_V2/external_data/大學基本資料.csv')

# 去除重複的經緯度
university_data.drop_duplicates(subset=['lat', 'lng'], inplace=True)

# 創建 BallTree
university_tree = BallTree(np.deg2rad(university_data[['lat', 'lng']].values), metric='haversine')

# 迭代 all_data DataFrame 計算距離
for idx, row in enumerate(all_data.itertuples()):
    # 使用 idx 從 coordinates 中獲取對應的經緯度
    lat = coordinates.loc[idx, 'lat']
    lng = coordinates.loc[idx, 'lng']

    # 新增四個特徵
    all_data.at[idx, 'num_universities_5km'] = count_facilities(university_tree, lat, lng, 5)
    all_data.at[idx, 'num_universities_10km'] = count_facilities(university_tree, lat, lng, 10)
    all_data.at[idx, 'distance_to_nearest_university'] = distance_to_nearest_facilities(university_tree, lat, lng)
    all_data.at[idx, 'average_distance_to_nearest_5_university'] = average_distance_to_nearest_facilities(university_tree, lat, lng)

# 查看更新後的 DataFrame
print(all_data[['num_universities_5km', 'num_universities_10km', 'distance_to_nearest_university', 'average_distance_to_nearest_5_university']].describe())



       num_universities_5km  num_universities_10km  \
count          23502.000000           23502.000000   
mean               4.845247              14.496596   
std                3.167643               7.672477   
min                0.000000               0.000000   
25%                2.000000               8.000000   
50%                4.000000              13.000000   
75%                7.000000              22.000000   
max               13.000000              31.000000   

       distance_to_nearest_university  \
count                    23502.000000   
mean                         2.238220   
std                          1.446047   
min                          0.015906   
25%                          1.158636   
50%                          1.933558   
75%                          3.032172   
max                         17.495319   

       average_distance_to_nearest_5_university  
count                              23502.000000  
mean                                   4.62

#### `公車站` : 1, 5 KM 內數量, 最近距離, 最近5平均距離

In [76]:
# 載入公車站點資料
bus_station_data = pd.read_csv('30_Training Dataset_V2/external_data/公車站點資料.csv')

# 去除重複的經緯度
bus_station_data.drop_duplicates(subset=['lat', 'lng'], inplace=True)

# 創建 BallTree
bus_station_tree = BallTree(np.deg2rad(bus_station_data[['lat', 'lng']].values), metric='haversine')

for idx, row in enumerate(all_data.itertuples()):
    # 使用 idx 從 coordinates 中獲取對應的經緯度
    lat = coordinates.loc[idx, 'lat']
    lng = coordinates.loc[idx, 'lng']

    # 新增四個特徵
    all_data.at[idx, 'num_bus_stations_1km'] = count_facilities(bus_station_tree, lat, lng, km=1)
    all_data.at[idx, 'num_bus_stations_5km'] = count_facilities(bus_station_tree, lat, lng, km=5)
    all_data.at[idx, 'distance_to_nearest_bus_stations'] = distance_to_nearest_facilities(bus_station_tree, lat, lng)
    all_data.at[idx, 'avg_distance_to_nearest_5_bus_stations'] = average_distance_to_nearest_facilities(bus_station_tree, lat, lng)

# 查看更新後的 DataFrame
print(all_data[['num_bus_stations_1km', 'num_bus_stations_5km', 'distance_to_nearest_bus_stations', 'avg_distance_to_nearest_5_bus_stations']].describe())


       num_bus_stations_1km  num_bus_stations_5km  \
count          23502.000000          23502.000000   
mean              70.060676           1168.702323   
std               30.591222            475.743135   
min                0.000000              0.000000   
25%               50.000000            840.000000   
50%               71.000000           1165.000000   
75%               90.000000           1525.000000   
max              179.000000           2125.000000   

       distance_to_nearest_bus_stations  \
count                      23502.000000   
mean                           0.927951   
std                           11.402375   
min                            0.000369   
25%                            0.063292   
50%                            0.101868   
75%                            0.153684   
max                          221.602953   

       avg_distance_to_nearest_5_bus_stations  
count                            23502.000000  
mean                                 1

#### `火車站` : 5, 10 KM 內數量, 最近距離, 最近5平均距離

In [77]:
# 載入火車站點資料
train_station_data = pd.read_csv('30_Training Dataset_V2/external_data/火車站點資料.csv')

# 去除重複的經緯度
train_station_data.drop_duplicates(subset=['lat', 'lng'], inplace=True)

# 創建 BallTree
train_station_tree = BallTree(np.deg2rad(train_station_data[['lat', 'lng']].values), metric='haversine')



for idx, row in enumerate(all_data.itertuples()):
    # 使用 idx 從 coordinates 中獲取對應的經緯度
    lat = coordinates.loc[idx, 'lat']
    lng = coordinates.loc[idx, 'lng']

    # 新增四個特徵
    all_data.at[idx, 'num_train_stations_5km'] = count_facilities(train_station_tree, lat, lng, km=5)
    all_data.at[idx, 'num_train_stations_10km'] = count_facilities(train_station_tree, lat, lng, km=10)
    all_data.at[idx, 'distance_to_nearest_train_stations'] = distance_to_nearest_facilities(train_station_tree, lat, lng)
    all_data.at[idx, 'avg_distance_to_nearest_5_train_stations'] = average_distance_to_nearest_facilities(train_station_tree, lat, lng)

# 查看更新後的 DataFrame
print(all_data[['num_train_stations_5km', 'num_train_stations_10km', 'distance_to_nearest_train_stations', 'avg_distance_to_nearest_5_train_stations']].describe())


       num_train_stations_5km  num_train_stations_10km  \
count            23502.000000             23502.000000   
mean                 2.961365                 7.382436   
std                  2.374145                 3.414419   
min                  0.000000                 0.000000   
25%                  1.000000                 5.000000   
50%                  3.000000                 7.000000   
75%                  4.000000                10.000000   
max                 11.000000                21.000000   

       distance_to_nearest_train_stations  \
count                        23502.000000   
mean                             3.969237   
std                             11.753643   
min                              0.014152   
25%                              1.393357   
50%                              2.515636   
75%                              4.380906   
max                            228.633828   

       avg_distance_to_nearest_5_train_stations  
count                

#### `金融機構` : 3, 5 KM 內數量, 最近距離, 最近5平均距離

In [78]:
# 載入金融機構基本資料
financial_data = pd.read_csv('30_Training Dataset_V2/external_data/金融機構基本資料.csv')

# 去除重複的經緯度
financial_data.drop_duplicates(subset=['lat', 'lng'], inplace=True)

# 創建 BallTree
financial_tree = BallTree(np.deg2rad(financial_data[['lat', 'lng']].values), metric='haversine')

for idx, row in enumerate(all_data.itertuples()):
    # 使用 idx 從 coordinates 中獲取對應的經緯度
    lat = coordinates.loc[idx, 'lat']
    lng = coordinates.loc[idx, 'lng']

    # 計算每個點附近的醫療機構數量
    all_data.at[idx, 'num_financial_institutions_3km'] = count_facilities(financial_tree, lat, lng, km=3)
    all_data.at[idx, 'num_financial_institutions_5km'] = count_facilities(financial_tree, lat, lng, km=5)
    all_data.at[idx, 'distance_to_nearest_financial_institutions'] = distance_to_nearest_facilities(financial_tree, lat, lng)
    all_data.at[idx, 'avg_distance_to_nearest_5_financial_institutions'] = average_distance_to_nearest_facilities(financial_tree, lat, lng)


# 查看更新後的 DataFrame
print(all_data[['num_financial_institutions_3km', 'num_financial_institutions_5km', 'distance_to_nearest_financial_institutions', 'avg_distance_to_nearest_5_financial_institutions']].describe())


       num_financial_institutions_3km  num_financial_institutions_5km  \
count                    23502.000000                    23502.000000   
mean                       107.280146                      247.879117   
std                        105.844375                      216.540073   
min                          0.000000                        1.000000   
25%                         38.000000                       85.000000   
50%                         76.000000                      180.000000   
75%                        136.000000                      322.000000   
max                        555.000000                      836.000000   

       distance_to_nearest_financial_institutions  \
count                                23502.000000   
mean                                     0.361187   
std                                      0.344575   
min                                      0.001045   
25%                                      0.138265   
50%                     

#### `便利商店` : 1, 2 KM 內數量, 最近距離, 最近5平均距離

In [79]:
# 載入便利商店資料
convenience_store_data = pd.read_csv('30_Training Dataset_V2/external_data/便利商店.csv')

# 去除經緯度為空的資料
convenience_store_data.dropna(subset=['lat', 'lng'], inplace=True)

# 去除重複的經緯度
convenience_store_data.drop_duplicates(subset=['lat', 'lng'], inplace=True)

# 創建 BallTree
convenience_store_tree = BallTree(np.deg2rad(convenience_store_data[['lat', 'lng']].values), metric='haversine')


for idx, row in enumerate(all_data.itertuples()):
    # 使用 idx 從 coordinates 中獲取對應的經緯度
    lat = coordinates.loc[idx, 'lat']
    lng = coordinates.loc[idx, 'lng']

    # 創建四個特徵
    all_data.at[idx, 'num_convenience_stores_1km'] = count_facilities(convenience_store_tree, lat, lng, km=1)
    all_data.at[idx, 'num_convenience_stores_2km'] = count_facilities(convenience_store_tree, lat, lng, km=2)
    all_data.at[idx, 'distance_to_nearest_convenience_stores'] = distance_to_nearest_facilities(convenience_store_tree, lat, lng)
    all_data.at[idx, 'avg_distance_to_nearest_5_convenience_stores'] = average_distance_to_nearest_facilities(convenience_store_tree, lat, lng)

# 查看更新後的 DataFrame
print(all_data[['num_convenience_stores_1km', 'num_convenience_stores_2km', 'distance_to_nearest_convenience_stores', 'avg_distance_to_nearest_5_convenience_stores']].describe())


       num_convenience_stores_1km  num_convenience_stores_2km  \
count                23502.000000                23502.000000   
mean                    39.789635                  128.603693   
std                     24.956685                   81.336967   
min                      0.000000                    0.000000   
25%                     21.000000                   68.000000   
50%                     34.000000                  110.000000   
75%                     53.000000                  176.000000   
max                    153.000000                  408.000000   

       distance_to_nearest_convenience_stores  \
count                            23502.000000   
mean                                 0.120765   
std                                  0.105634   
min                                  0.000751   
25%                                  0.060421   
50%                                  0.097894   
75%                                  0.150287   
max                   

#### `科學園區` : 最近的距離

In [80]:
science_park_data = pd.read_csv('30_Training Dataset_V2/external_data/科技工業園區.csv')

# 轉換成 WGS84 經緯度
transformer = Transformer.from_crs("epsg:3826", "epsg:4326", always_xy=True)
science_park_data['lng'], science_park_data['lat'] = transformer.transform(science_park_data['X'].values, science_park_data['Y'].values)

# 去除重複的經緯度
science_park_data.drop_duplicates(subset=['lat', 'lng'], inplace=True)

# 創建 BallTree
science_park_tree = BallTree(np.deg2rad(science_park_data[['lat', 'lng']].values), metric='haversine')


for idx, row in enumerate(all_data.itertuples()):
    lat = coordinates.loc[idx, 'lat']
    lng = coordinates.loc[idx, 'lng']

    # 新增特徵
    all_data.at[idx, 'distance_to_nearest_science_park'] = distance_to_nearest_facilities(science_park_tree, lat, lng)

# 查看更新後的 DataFrame
print(all_data['distance_to_nearest_science_park'].describe())



count    23502.000000
mean         4.963971
std         10.441665
min          0.127250
25%          2.437649
50%          3.976379
75%          6.017188
max        205.651579
Name: distance_to_nearest_science_park, dtype: float64


#### `殯葬設施` : 最近的距離

In [81]:
cremation_ground_data = pd.read_csv('30_Training Dataset_V2/external_data/火葬場.csv')

# 去除重複的經緯度
cremation_ground_data.drop_duplicates(subset=['lat', 'lng'], inplace=True)

# 創建 BallTree
cremation_ground_tree = BallTree(np.deg2rad(cremation_ground_data[['lat', 'lng']].values), metric='haversine')


for idx, row in enumerate(all_data.itertuples()):
    lat = coordinates.loc[idx, 'lat']
    lng = coordinates.loc[idx, 'lng']

    # 新增特徵
    all_data.at[idx, 'distance_to_nearest_cremation_ground'] = distance_to_nearest_facilities(cremation_ground_tree, lat, lng)

# 查看更新後的 DataFrame
print(all_data['distance_to_nearest_cremation_ground'].describe())



count    23502.000000
mean         8.771742
std          4.460561
min          0.151749
25%          4.993487
50%          8.259829
75%         12.295524
max         26.384640
Name: distance_to_nearest_cremation_ground, dtype: float64


#### `商圈` : 最近的距離

In [97]:
market_data = pd.read_csv('30_Training Dataset_V2/external_data/商圈.csv')

# 去除重複的經緯度
market_data.drop_duplicates(subset=['lat', 'lng'], inplace=True)

# 創建 BallTree
market_tree = BallTree(np.deg2rad(market_data[['lat', 'lng']].values), metric='haversine')


for idx, row in enumerate(all_data.itertuples()):
    lat = coordinates.loc[idx, 'lat']
    lng = coordinates.loc[idx, 'lng']

    # 新增特徵
    all_data.at[idx, 'avg_distance_to_nearest_5_high_schools'] = average_distance_to_nearest_facilities(market_tree, lat, lng)

# 查看更新後的 DataFrame
print(all_data['distance_to_nearest_market'].describe())



count    23502.000000
mean         2.105255
std          1.873126
min          0.001061
25%          0.894978
50%          1.651621
75%          2.767918
max         21.158972
Name: distance_to_nearest_market, dtype: float64


#### `高中` : 3, 5 KM 內數量, 最近距離, 最近5平均距離

In [83]:
# 載入高中基本資料
high_school_data = pd.read_csv('30_Training Dataset_V2/external_data/高中基本資料.csv')

# 去除重複的經緯度
high_school_data.drop_duplicates(subset=['lat', 'lng'], inplace=True)

# 創建 BallTree
high_school_tree = BallTree(np.deg2rad(high_school_data[['lat', 'lng']].values), metric='haversine')

for idx, row in enumerate(all_data.itertuples()):
    # 使用 idx 從 coordinates 中獲取對應的經緯度
    lat = coordinates.loc[idx, 'lat']
    lng = coordinates.loc[idx, 'lng']

    # 新增四個特徵
    all_data.at[idx, 'num_high_schools_3km'] = count_facilities(high_school_tree, lat, lng, km=3)
    all_data.at[idx, 'num_high_schools_5km'] = count_facilities(high_school_tree, lat, lng, km=5)
    all_data.at[idx, 'distance_to_nearest_high_schools'] = distance_to_nearest_facilities(high_school_tree, lat, lng)
    all_data.at[idx, 'avg_distance_to_nearest_5_high_schools'] = average_distance_to_nearest_facilities(high_school_tree, lat, lng)


# 查看更新後的 DataFrame
print(all_data[['num_high_schools_3km', 'num_high_schools_5km', 'distance_to_nearest_high_schools', 'avg_distance_to_nearest_5_high_schools']].describe())

       num_high_schools_3km  num_high_schools_5km  \
count          23502.000000          23502.000000   
mean               7.107438             17.049315   
std                5.015561             11.329539   
min                0.000000              0.000000   
25%                3.000000              8.000000   
50%                6.000000             15.000000   
75%               10.000000             23.000000   
max               24.000000             47.000000   

       distance_to_nearest_high_schools  \
count                      23502.000000   
mean                           1.045091   
std                            0.785088   
min                            0.000844   
25%                            0.509399   
50%                            0.831900   
75%                            1.344526   
max                            8.555398   

       avg_distance_to_nearest_5_high_schools  
count                            23502.000000  
mean                                 2

#### `國小` : 3, 5 KM 內數量, 最近距離, 最近5平均距離

In [84]:
# 載入國小基本資料
elementary_school_data = pd.read_csv('30_Training Dataset_V2/external_data/國小基本資料.csv')

# 去除重複的經緯度
elementary_school_data.drop_duplicates(subset=['lat', 'lng'], inplace=True)

# 創建 BallTree
elementary_school_tree = BallTree(np.deg2rad(elementary_school_data[['lat', 'lng']].values), metric='haversine')

for idx, row in enumerate(all_data.itertuples()):
    # 使用 idx 從 coordinates 中獲取對應的經緯度
    lat = coordinates.loc[idx, 'lat']
    lng = coordinates.loc[idx, 'lng']

    # 新增四個特徵
    all_data.at[idx, 'num_elementary_schools_3km'] = count_facilities(elementary_school_tree, lat, lng, km=3)
    all_data.at[idx, 'num_elementary_schools_5km'] = count_facilities(elementary_school_tree, lat, lng, km=5)
    all_data.at[idx, 'distance_to_nearest_elementary_schools'] = distance_to_nearest_facilities(elementary_school_tree, lat, lng)
    all_data.at[idx, 'avg_distance_to_nearest_5_elementary_schools'] = average_distance_to_nearest_facilities(elementary_school_tree, lat, lng)


# 查看更新後的 DataFrame
print(all_data[['num_elementary_schools_3km', 'num_elementary_schools_5km', 'distance_to_nearest_elementary_schools', 'avg_distance_to_nearest_5_elementary_schools']].describe())

       num_elementary_schools_3km  num_elementary_schools_5km  \
count                23502.000000                23502.000000   
mean                    16.112373                   39.364565   
std                      8.464501                   20.511553   
min                      1.000000                    2.000000   
25%                     10.000000                   22.000000   
50%                     14.000000                   36.000000   
75%                     21.000000                   53.000000   
max                     43.000000                   91.000000   

       distance_to_nearest_elementary_schools  \
count                            23502.000000   
mean                                 0.512435   
std                                  0.308855   
min                                  0.010019   
25%                                  0.296748   
50%                                  0.446629   
75%                                  0.646383   
max                   

#### `國中` : 3, 5 KM 內數量, 最近距離, 最近5平均距離

In [85]:
# 載入國中基本資料
junior_high_school_data = pd.read_csv('30_Training Dataset_V2/external_data/國中基本資料.csv')

# 去除重複的經緯度
junior_high_school_data.drop_duplicates(subset=['lat', 'lng'], inplace=True)

# 創建 BallTree
junior_high_school_tree = BallTree(np.deg2rad(junior_high_school_data[['lat', 'lng']].values), metric='haversine')


for idx, row in enumerate(all_data.itertuples()):
    # 使用 idx 從 coordinates 中獲取對應的經緯度
    lat = coordinates.loc[idx, 'lat']
    lng = coordinates.loc[idx, 'lng']

    # 新增四個特徵
    all_data.at[idx, 'num_junior_high_schools_3km'] = count_facilities(junior_high_school_tree, lat, lng, km=3)
    all_data.at[idx, 'num_junior_high_schools_5km'] = count_facilities(junior_high_school_tree, lat, lng, km=5)
    all_data.at[idx, 'distance_to_nearest_junior_high_schools'] = distance_to_nearest_facilities(junior_high_school_tree, lat, lng)
    all_data.at[idx, 'avg_distance_to_nearest_5_junior_high_schools'] = average_distance_to_nearest_facilities(junior_high_school_tree, lat, lng)


# 查看更新後的 DataFrame
print(all_data[['num_junior_high_schools_3km', 'num_junior_high_schools_5km', 'distance_to_nearest_junior_high_schools', 'avg_distance_to_nearest_5_junior_high_schools']].describe())

       num_junior_high_schools_3km  num_junior_high_schools_5km  \
count                 23502.000000                 23502.000000   
mean                      9.808101                    23.400604   
std                       5.352231                    12.728366   
min                       0.000000                     0.000000   
25%                       6.000000                    13.000000   
50%                       9.000000                    22.000000   
75%                      13.000000                    31.000000   
max                      27.000000                    55.000000   

       distance_to_nearest_junior_high_schools  \
count                             23502.000000   
mean                                  0.680651   
std                                   0.414131   
min                                   0.011246   
25%                                   0.389820   
50%                                   0.593588   
75%                                   0.870774

#### `捷運站` : 3, 5 KM 內數量, 最近距離, 最近5平均距離

In [86]:
# 載入捷運站點資料
metro_data = pd.read_csv('30_Training Dataset_V2/external_data/捷運站點資料.csv')

# 去除重複的經緯度
metro_data.drop_duplicates(subset=['lat', 'lng'], inplace=True)

# 創建 BallTree
metro_tree = BallTree(np.deg2rad(metro_data[['lat', 'lng']].values), metric='haversine')


for idx, row in enumerate(all_data.itertuples()):
    # 使用 idx 從 coordinates 中獲取對應的經緯度
    lat = coordinates.loc[idx, 'lat']
    lng = coordinates.loc[idx, 'lng']

    # 新增四個特徵
    all_data.at[idx, 'num_metros_3km'] = count_facilities(metro_tree, lat, lng, km=3)
    all_data.at[idx, 'num_metros_5km'] = count_facilities(metro_tree, lat, lng, km=5)
    all_data.at[idx, 'distance_to_nearest_metrons'] = distance_to_nearest_facilities(metro_tree, lat, lng)
    all_data.at[idx, 'avg_distance_to_nearest_5_metros'] = average_distance_to_nearest_facilities(metro_tree, lat, lng)


# 查看更新後的 DataFrame
print(all_data[['num_metros_3km', 'num_metros_5km', 'distance_to_nearest_metrons', 'avg_distance_to_nearest_5_metros']].describe())

       num_metros_3km  num_metros_5km  distance_to_nearest_metrons  \
count    23502.000000    23502.000000                 23502.000000   
mean         7.471109       17.865288                    14.960486   
std          7.889923       17.305613                    33.228299   
min          0.000000        0.000000                     0.006895   
25%          0.000000        0.000000                     0.492547   
50%          5.000000       15.000000                     1.098174   
75%         12.000000       31.000000                     7.728923   
max         36.000000       57.000000                   275.844520   

       avg_distance_to_nearest_5_metros  
count                      23502.000000  
mean                          16.120550  
std                           33.771758  
min                            0.461946  
25%                            1.133731  
50%                            1.846617  
75%                            8.848332  
max                          278.

#### `郵局` : 3, 5 KM 內數量, 最近距離, 最近5平均距離

In [87]:
# 載入郵局據點資料
post_office_data = pd.read_csv('30_Training Dataset_V2/external_data/郵局據點資料.csv')

# 去除重複的經緯度
post_office_data.drop_duplicates(subset=['lat', 'lng'], inplace=True)

# 創建 BallTree
post_office_tree = BallTree(np.deg2rad(post_office_data[['lat', 'lng']].values), metric='haversine')

for idx, row in enumerate(all_data.itertuples()):
    # 使用 idx 從 coordinates 中獲取對應的經緯度
    lat = coordinates.loc[idx, 'lat']
    lng = coordinates.loc[idx, 'lng']

    # 創建四個特徵
    all_data.at[idx, 'num_post_offices_3km'] = count_facilities(post_office_tree, lat, lng, km=3)
    all_data.at[idx, 'num_post_offices_5km'] = count_facilities(post_office_tree, lat, lng, km=5)
    all_data.at[idx, 'distance_to_nearest_post_offices'] = distance_to_nearest_facilities(post_office_tree, lat, lng)
    all_data.at[idx, 'avg_distance_to_nearest_5_post_offices'] = average_distance_to_nearest_facilities(post_office_tree, lat, lng)


# 查看更新後的 DataFrame
print(all_data[['num_post_offices_3km', 'num_post_offices_5km', 'distance_to_nearest_post_offices', 'avg_distance_to_nearest_5_post_offices']].describe())

       num_post_offices_3km  num_post_offices_5km  \
count          23502.000000          23502.000000   
mean              17.809506             42.789550   
std               12.157432             28.089489   
min                0.000000              1.000000   
25%                8.000000             21.000000   
50%               15.000000             36.000000   
75%               24.000000             60.000000   
max               62.000000            109.000000   

       distance_to_nearest_post_offices  \
count                      23502.000000   
mean                           0.514880   
std                            0.350013   
min                            0.002185   
25%                            0.280179   
50%                            0.445300   
75%                            0.657847   
max                            4.087681   

       avg_distance_to_nearest_5_post_offices  
count                            23502.000000  
mean                                 1

#### `腳踏車站` : 3, 5 KM 內數量, 最近距離, 最近5平均距離

In [88]:
# 載入腳踏車站點資料
bike_station_data = pd.read_csv('30_Training Dataset_V2/external_data/腳踏車站點資料.csv')

# 去除重複的經緯度
bike_station_data.drop_duplicates(subset=['lat', 'lng'], inplace=True)

# 創建 BallTree
bike_station_tree = BallTree(np.deg2rad(bike_station_data[['lat', 'lng']].values), metric='haversine')

for idx, row in enumerate(all_data.itertuples()):
    # 使用 idx 從 coordinates 中獲取對應的經緯度
    lat = coordinates.loc[idx, 'lat']
    lng = coordinates.loc[idx, 'lng']

    # 創建四個特徵：
    all_data.at[idx, 'num_bike_stations_3km'] = count_facilities(bike_station_tree, lat, lng, km=3)
    all_data.at[idx, 'num_bike_stations_5km'] = count_facilities(bike_station_tree, lat, lng, km=5)
    all_data.at[idx, 'distance_to_nearest_bike_stations'] = distance_to_nearest_facilities(bike_station_tree, lat, lng)
    all_data.at[idx, 'avg_distance_to_nearest_5_bike_stations'] = average_distance_to_nearest_facilities(bike_station_tree, lat, lng)


# 查看更新後的 DataFrame
print(all_data[['num_bike_stations_3km', 'num_bike_stations_5km', 'distance_to_nearest_bike_stations', 'avg_distance_to_nearest_5_bike_stations']].describe())

       num_bike_stations_3km  num_bike_stations_5km  \
count           23502.000000           23502.000000   
mean              165.456685             393.434729   
std               118.257622             277.185524   
min                 0.000000               0.000000   
25%                64.000000             131.000000   
50%               157.000000             393.000000   
75%               262.000000             617.000000   
max               493.000000             986.000000   

       distance_to_nearest_bike_stations  \
count                       23502.000000   
mean                            1.290871   
std                            11.734327   
min                             0.000210   
25%                             0.099761   
50%                             0.156688   
75%                             0.240080   
max                           224.081617   

       avg_distance_to_nearest_5_bike_stations  
count                             23502.000000  
mean     

#### `醫療機構` : 3, 5 KM 內數量, 最近距離, 最近5平均距離

In [89]:
# 載入醫療機構資料
medical_data = pd.read_csv('30_Training Dataset_V2/external_data/醫療機構基本資料.csv')

# 去除重複的經緯度
medical_data.drop_duplicates(subset=['lat', 'lng'], inplace=True)

# 創建 BallTree
medical_tree = BallTree(np.deg2rad(medical_data[['lat', 'lng']].values), metric='haversine')

for idx, row in enumerate(all_data.itertuples()):
    # 使用 idx 從 coordinates 中獲取對應的經緯度
    lat = coordinates.loc[idx, 'lat']
    lng = coordinates.loc[idx, 'lng']

    # 創建四個特徵
    all_data.at[idx, 'num_med_3km'] = count_facilities(medical_tree, lat, lng, km=3)
    all_data.at[idx, 'num_med_5km'] = count_facilities(medical_tree, lat, lng, km=5)
    all_data.at[idx, 'distance_to_nearest_medical_institutions'] = distance_to_nearest_facilities(medical_tree, lat, lng)
    all_data.at[idx, 'avg_distance_to_nearest_5_medical_institutions'] = average_distance_to_nearest_facilities(medical_tree, lat, lng)


# 查看更新後的 DataFrame
print(all_data[['num_med_3km', 'num_med_5km', 'distance_to_nearest_medical_institutions', 'avg_distance_to_nearest_5_medical_institutions']].describe())


        num_med_3km   num_med_5km  distance_to_nearest_medical_institutions  \
count  23502.000000  23502.000000                              23502.000000   
mean     484.993235   1106.613565                                  0.168088   
std      366.133137    770.520228                                  0.209053   
min        0.000000      4.000000                                  0.000589   
25%      175.000000    436.000000                                  0.055113   
50%      405.000000   1004.000000                                  0.103925   
75%      721.000000   1660.000000                                  0.197084   
max     1705.000000   3000.000000                                  4.397377   

       avg_distance_to_nearest_5_medical_institutions  
count                                    23502.000000  
mean                                         0.270635  
std                                          0.267980  
min                                          0.017167  
25%     

#### `ATM` : 2, 5 KM 內數量, 最近距離, 最近5平均距離

In [90]:
# 載入ATM資料
ATM_data = pd.read_csv('30_Training Dataset_V2/external_data/ATM資料.csv')

# 去除重複的經緯度
ATM_data.drop_duplicates(subset=['lat', 'lng'], inplace=True)

# 創建 BallTree
ATM_tree = BallTree(np.deg2rad(ATM_data[['lat', 'lng']].values), metric='haversine')

for idx, row in enumerate(all_data.itertuples()):
    # 使用 idx 從 coordinates 中獲取對應的經緯度
    lat = coordinates.loc[idx, 'lat']
    lng = coordinates.loc[idx, 'lng']

    # 創建四個特徵
    all_data.at[idx, 'num_ATMs_2km'] = count_facilities(ATM_tree, lat, lng, km=2)
    all_data.at[idx, 'num_ATMs_5km'] = count_facilities(ATM_tree, lat, lng, km=5)
    all_data.at[idx, 'distance_to_nearest_ATM'] = distance_to_nearest_facilities(ATM_tree, lat, lng)
    all_data.at[idx, 'avg_distance_to_nearest_5_ATMs'] = average_distance_to_nearest_facilities(ATM_tree, lat, lng)


# 查看更新後的 DataFrame
print(all_data[['num_ATMs_2km', 'num_ATMs_5km', 'distance_to_nearest_ATM', 'avg_distance_to_nearest_5_ATMs']].describe())




       num_ATMs_2km  num_ATMs_5km  distance_to_nearest_ATM  \
count  23502.000000  23502.000000             23502.000000   
mean     197.666241    930.490384                 0.109442   
std      140.384365    645.774031                 0.098133   
min        0.000000      7.000000                 0.000396   
25%       96.000000    431.000000                 0.053562   
50%      164.000000    737.000000                 0.088090   
75%      261.000000   1359.000000                 0.136973   
max      746.000000   2513.000000                 2.074883   

       avg_distance_to_nearest_5_ATMs  
count                    23502.000000  
mean                         0.210874  
std                          0.139694  
min                          0.014224  
25%                          0.133047  
50%                          0.180698  
75%                          0.248576  
max                          2.447058  


### `model 3` : Label + hash + One-hot encoding + 新增特徵

In [91]:
X = all_data[:train.shape[0]]
X_test = all_data[train.shape[0]:]
y = train["單價"]
print("X shape is {}".format(X.shape))
print("X_test shape is {}".format(X_test.shape))

X shape is (11751, 481)
X_test shape is (11751, 481)


In [92]:
%%time

model = xgb.XGBRegressor(random_state=42)
model_score = cross_val_score(model, X, y, scoring='neg_mean_absolute_percentage_error', cv=5)

print(model_score)

model_score = -1 * model_score.mean()

print("MAPE: %.4f%%" % (model_score * 100.0))

[-0.09924835 -0.09437245 -0.09602771 -0.1002802  -0.09995889]
MAPE: 9.7978%
CPU times: user 50.4 s, sys: 3.91 s, total: 54.3 s
Wall time: 10.8 s


lower bound : 9.6585%


### 訓練模型

參數搜尋

In [93]:
from sklearn.model_selection import GridSearchCV

xgb_reg = xgb.XGBRegressor()

# 定義參數網格
param_grid = {
    'n_estimators': [100, 500, 1000],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'colsample_bytree': [0.5, 0.7, 1],
    'subsample': [0.5, 0.7, 1]
}

grid_search = GridSearchCV(estimator=xgb_reg, param_grid=param_grid, 
                           cv=3, n_jobs=-1, verbose=2, scoring='neg_mean_absolute_percentage_error')

#grid_search.fit(X, y)

#best_params = {'colsample_bytree': 0.5, 'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 1000, 'subsample': 1}

# 印出最佳參數
#print("Best parameters:", grid_search.best_params_)
#print("Best score:", grid_search.best_score_)

`Best parameters: {'colsample_bytree': 0.5, 'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 1000, 'subsample': 1}`

`Best score: -0.08904505796119107`

In [94]:
%%time

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error

sum = 0
params = {'learning_rate': 0.05, 'n_estimators': 1000}

for _ in range(5):
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

    xgb_model = xgb.XGBRegressor(**params, early_stopping_rounds=5, n_jobs=-1, random_state=42)
    xgb_model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=False) # verbose=True: print evaluation result

    y_pred = xgb_model.predict(X_valid)
    mape = mean_absolute_percentage_error(y_valid, y_pred)
    sum += mape
    print("MAPE: %.4f%%" % (mape * 100.0))



print("Average MAPE: %.4f%%" % (sum * 100.0 / 5))


MAPE: 9.3465%
MAPE: 9.3496%
MAPE: 9.1613%
MAPE: 8.7791%
MAPE: 9.2371%
Average MAPE: 9.1747%
CPU times: user 4min 9s, sys: 15.7 s, total: 4min 25s
Wall time: 52.8 s


lower bound : 9.0838%

In [95]:
importances = xgb_model.feature_importances_
feature_names = X.columns

# 將特徵名稱和重要性配對，並按重要性排序
feature_importances = sorted(zip(feature_names, importances), key=lambda x: x[1], reverse=True)

# 印出排序後的特徵重要性
for name, importance in feature_importances:
    print(f'{name}: {importance}')


num_financial_institutions_5km: 0.3765760362148285
num_med_3km: 0.09681439399719238
num_universities_10km: 0.04512835666537285
num_financial_institutions_3km: 0.044807955622673035
num_junior_high_schools_5km: 0.041831132024526596
鄉鎮市區_39: 0.030012212693691254
橫坐標: 0.0287938229739666
主要建材_鋼骨造: 0.02190745621919632
鄉鎮市區_126: 0.018868159502744675
鄉鎮市區_44: 0.0173215102404356
屋齡: 0.012782514095306396
num_bike_stations_3km: 0.012438197620213032
路名_167: 0.012348701246082783
縣市: 0.011481139808893204
鄉鎮市區_67: 0.008973832242190838
distance_to_nearest_cremation_ground: 0.008067695423960686
num_bike_stations_5km: 0.008033906109631062
鄉鎮市區_125: 0.007214385084807873
總樓層數: 0.006824818439781666
num_post_offices_3km: 0.004970177076756954
主要建材_鋼筋混凝土造: 0.004726414103060961
distance_to_nearest_science_park: 0.004643010441213846
distance_to_nearest_metrons: 0.004558248445391655
avg_distance_to_nearest_5_metros: 0.004365652799606323
avg_distance_to_nearest_5_financial_institutions: 0.004075620789080858
鄉鎮市區_

進行預測

In [96]:
# 將預測結果存成csv檔
# 把 public test 與 private test 串接起來
test_ID = pd.concat([public_test_ID, private_test_ID])

y_submit = xgb_model.predict(X_test)
submit = pd.DataFrame(y_submit, columns=['predicted_price'])
submit.index = test_ID
submit.index.name = 'ID'

# add time stamp to file name
import time
timestr = time.strftime("%Y%m%d-%H%M%S")
# uncomment to save file
submit.to_csv('./submit/submit_' + timestr + '.csv')
submit.head(10)

,predicted_price
ID,
PU-1,1.723103
PU-2,1.690139
PU-3,2.780596
PU-4,1.538442
PU-5,3.657236
PU-6,1.658502
PU-7,1.870648
PU-8,1.727861
PU-9,1.244269
